# 8. Higgsin bosoni

Tässä harjoituksessa tutustutaan siihen, miten [Higgsin bosonin](https://fi.wikipedia.org/wiki/Higgsin_bosoni) olemassaolo todistettiin. Standardimallin ennusteiden mukaan Higgsin bosoni voi hajota siten, että siitä syntyy ensin kaksi Z-bosonia ja niistä edelleen neljä [leptonia](https://fi.wikipedia.org/wiki/Leptoni) (ks. kuva 1). Se ei ole ainoa prosessi jolla on samanlainen lopputila, joten meidän täytyy löytää kaikkien tapahtumien joukosta ne, joissa Higgs esiintyy. Teoria itsessään ei kerro paljoakaan siitä, mikä Higgsin hiukkasen massa voisi olla, mutta esimerkiksi neljään leptoniin hajoaminen on todennäköisempää tietyillä massoilla, joihin voimme aluksi kohdistaa etsintämme.

![](https://github.com/cms-opendata-education/cms-jupyter-materials-finnish/blob/master/Kuvat/higgs-4l.png?raw=true)
*Higgsin bosonin hajoaminen neljäksi leptoniksi*

Tarkoituksenamme onkin tehdä histogrammi CMS-ilmaisimen tuottamasta datasta vuosilta 2011-2012 sellaisista törmäyksistä, joissa on syntynyt 4 leptonia. 

Ennen kuin alamme tutkimaan varsinaisia mittaustuloksia, käytetään simulaatioista saatuja arvoja luodaksemme ennuste, miltä histogrammin pitäisi näyttää. Koska neljä leptonia voi syntyä useilla eri tavoilla, tulee tapaukset käsitellä erikseen. Alapuolella olevaan koodisoluun on laskettu [Monte Carlo](https://fi.wikipedia.org/wiki/Monte_Carlo_-simulaatio) -menetelmällä tuotettuja arvoja, jotka vastaavat törmäyksessä tapahtuvia tunnettuja prosesseja.

In [ ]:
# Tuodaan tarvittavat moduulit.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Simuloidut arvot on laskettu invarianttien massojen välille 70-181 GeV.

# Simuloidut arvot Z-bosonin ja jettien hajoamisesta neljäksi leptoniksi.
dy = np.array([0,0,0,0,0,0.354797,0.177398,2.60481,0,0,0,0,0,0,0,0,0,0.177398,0.177398,0,0.177398,0,0,0,0,0,0,0,0,0,0,0,0.177398,0,0,0,0])

# Simuloidut arvot huippukvarkin ja antihuippukvarkin hajoamiseen neljäksi leptoniksi.
ttbar = np.array([0.00465086,0,0.00465086,0,0,0,0,0,0,0,0.00465086,0,0,0,0,0,0.00465086,0,0,0,0,0.00465086,0.00465086,0,0,0.0139526,0,0,0.00465086,0,0,0,0.00465086,0.00465086,0.0139526,0,0])

# Simuloidut arvot kahden Z-bosonin hajoamiseen neljäksi leptoniksi.
zz = np.array([0.181215,0.257161,0.44846,0.830071,1.80272,4.57354,13.9677,14.0178,4.10974,1.58934,0.989974,0.839775,0.887188,0.967021,1.07882,1.27942,1.36681,1.4333,1.45141,1.41572,1.51464,1.45026,1.47328,1.42899,1.38757,1.33561,1.3075,1.29831,1.31402,1.30672,1.36442,1.39256,1.43472,1.58321,1.85313,2.19304,2.95083])

# Simuloidut arvot Higgsin bosonin hajoamiseen kahdeksi Z-bosoniksi, jotka hajoavat neljäksi leptoniksi.
# Oletuksena, että Higgsin massa on 125 GeV.
hzz = np.array([0.00340992,0.00450225,0.00808944,0.0080008,0.00801578,0.0108945,0.00794274,0.00950757,0.0130648,0.0163568,0.0233832,0.0334813,0.0427229,0.0738129,0.13282,0.256384,0.648352,2.38742,4.87193,0.944299,0.155005,0.0374193,0.0138906,0.00630364,0.00419265,0.00358719,0.00122527,0.000885718,0.000590479,0.000885718,0.000797085,8.86337e-05,0.000501845,8.86337e-05,0.000546162,4.43168e-05,8.86337e-05])

In [ ]:
# Luetaan datatiedostot valmiiksi myöhempää käyttöä varten. 
# Data on jaettu eri tiedostoihin sen perusteella, miltä vuodelta se on sekä mitä hiukkasia on syntynyt.
# Esimerkiksi tiedosto 4mu_2011.csv sisältää vuonna 2011 mitattuja tapahtumia, joissa on syntynyt 4 myonia.

csvs = [pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/4mu_2011.csv'),
        pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/4e_2011.csv'), 
        pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/2e2mu_2011.csv')]
csvs += [pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/4mu_2012.csv'),
         pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/4e_2012.csv'), 
         pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/2e2mu_2012.csv')]

# Yhdistetään csv-tiedostot yhdeksi ketjuksi.
fourlep = pd.concat(csvs)

In [ ]:
# Määritellään histogrammien piirtoa varten
#     rmin, rmax: minimi- ja maksimialueet invariantille massalle.
#     nbins:      pylväiden lukumäärä
#     width:      pylväiden leveys
#     center:     pylvään keskikohta
# Tutkitaan invarianttien massojen aluetta välillä 70-181 GeV. Valitaan pylväiden määräksi mielivaltaisesti 37.

rmin = 70
rmax = 181
nbins = 37

M_hist = np.histogram(fourlep['M'], bins = nbins, range = (rmin,rmax))
hist, bins = M_hist

width = 1.0*(bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2

In [ ]:
# Piirretään pylväsdiagrammi simuloiduista arvoista, joissa kaksi Z-bosonia hajoaa neljäksi leptoniksi.

plt.figure(figsize = (15,3))
# Alla olevassa koodissa 'zz' määrittää, mitkä simuloidut arvot haluamme piirtää.
plt.bar(center, zz, align = 'center', width = width, color = 'b', linewidth = 0, edgecolor = 'black', alpha = 0.5)
plt.xlabel('Invariantti massa (GeV)', fontsize = 15)
plt.ylabel('Tapahtumia', fontsize = 15)
plt.xlim(rmin,rmax)
plt.show()

> Miltä muut simuloidut tapaukset näyttävät?
<br>
$\color{purple}{\text{Kirjoita koodi alle}}$

In [ ]:
# Piirrä pylväsdiagrammi simuloiduista arvoista, joissa yksi Z-bosoni ja jetit hajoavat neljäksi leptoniksi (dy).
# Käytä pylväsdiagrammissa jotain muuta väriä kuin aiemmin, esimerkiksi vihreä 'g' tai harmaa 'grey'.



In [ ]:
# Piirrä pylväsdiagrammi simuloiduista arvoista, joissa huippu- ja antihuippukvarkki hajoavat neljäksi leptoniksi (ttbar).
# Käytä pylväsdiagrammissa jotain muuta väriä kuin aiemmin, esimerkiksi vihreä 'g' tai harmaa 'grey'.



Saadaksemme ennusteelle kokonaiskuvan, piirretään äsken piirtämämme pylväsdiagrammit samaan kuvaan. Tämä ennuste ei ota Higgsin bosonia huomioon. Voit ajaa koodin suoraan.

In [ ]:
plt.figure(figsize = (15,5))

# ttbar
tt_bar = plt.bar(center, ttbar, align = 'center', width = width, color = 'grey', linewidth = 0, edgecolor = 'black',
                 alpha = 0.5, label = r'ttbar $\rightarrow$ 4l')

# DY
dy_bar = plt.bar(center, dy, align = 'center', width = width, color = 'g', linewidth = 0, edgecolor = 'black',
                 alpha = 0.5, bottom = ttbar, label = r'dy $\rightarrow$ 4l')

# ZZ
zz_bar = plt.bar(center, zz, align = 'center', width = width, color = 'b', linewidth = 0, edgecolor = 'black',
                 alpha = 0.5, bottom = ttbar+dy, label = r'ZZ $\rightarrow$ 4l')


plt.xlabel('Invariantti massa (GeV)', fontsize = 15)
plt.ylabel('Tapahtumia', fontsize = 15)
plt.ylim(0,20)
plt.xlim(rmin,rmax)
plt.legend(fontsize = 15)

plt.show()

Millaisen ennusteen voit tehdä saamasi kuvaajan perusteella?
Tutkitaan, miltä kuvaaja näyttää, jos lisäämme siihen oikeat mittaukset.

In [ ]:
# Piirretään oikea data ja simulaatiot samaan kuvaan.

plt.figure(figsize = (15,5))

# Virherajat
xerrs = [width*0.5 for i in range(0, nbins)]
yerrs = np.sqrt(hist)

# ZZ
zz_bar = plt.bar(center, zz, align = 'center', width = width, color = 'b', linewidth = 0, edgecolor = 'black',
                 alpha = 0.5, bottom = ttbar+dy, label = r'ZZ')

# Mittaukset.
data_bar = plt.errorbar(center, hist, xerr = xerrs, yerr = yerrs, linestyle = 'None', color = 'black',
                        marker = 'o', label = 'Data')
# DY
dy_bar = plt.bar(center, dy, align = 'center', width = width, color = 'g', linewidth = 0, edgecolor = 'black',
                 alpha = 0.5, bottom = ttbar, label = r'dy $\rightarrow$ 4l')

# ttbar
tt_bar = plt.bar(center, ttbar, align = 'center', width = width, color = 'grey', linewidth = 0, edgecolor = 'black',
                 alpha = 0.5, label = r'ttbar $\rightarrow$ 4l')

plt.title('$ \sqrt{s} = 7$ TeV, L = 2.3 $fb^{-1}$; $\sqrt{s} = 8$ TeV, L = 11.6 $fb^{-1}$ \n', fontsize = 12)
plt.xlabel('Invariantti massa (GeV)', fontsize = 15)
plt.ylabel('Tapahtumia', fontsize = 15)
plt.ylim(0,25)
plt.xlim(rmin,rmax)
plt.legend(fontsize = 15)

plt.show()

> Vastaako data ennustetta?
<br>
> Miltä kuvaaja näyttäisi jos lisäisimme siihen ennusteen Higgsin bosonista? Simuloidut arvot on tallennettu muuttujaan 'hzz'.
<br>
> $\color{purple}{\text{Kirjoita koodi}}$

In [ ]:
# Piirrä pylväsdiagrammit kaikista simulaatioista mukaan lukien Higgsin bosonin simulaatio, 
# sekä lisää mukaan vielä oikea mittausdata.



> Mitä voit päätellä tuloksista?

Alla olevassa kuvassa on CMS:n analyysi törmäyksistä, joissa on syntynyt neljä leptonia.
> Vastaavatko saamasi tulokset CMS:n analyysiä?

<img src = 'https://inspirehep.net/record/1124338/files/H4l_mass_v3.png' align = 'right'>
